#### The following Jupyter Notebook is an adaptation of the 'Figure_4_Plots.ipnyb' Jupyter Notebook. We redo the same analysis, but randomizing the severe / non severe assignments of each patient.

In [1]:
setwd('../')


In [14]:
set.seed(2023) #for reproducibility

#### Setup for severe - non-severe patients

In [3]:
phenotype <- "severe" # "severe" for severe-only analysis, "notsevere" for non-severe genes.

destfile= "Plots/Figure4_" #Directory destination and name of the csv and pdf files that will be retrieved as output.

 
#libraries needed:
library("AnnotationDbi")
library("AnnotationDbi")
library("org.Hs.eg.db")
library("igraph")
library('parallelDist')
library('reshape2')

Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain 

### Read Community structure output files from MolTi

In [4]:
cero <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution0MultilayerNetwork5112018.csv")
cero <- strsplit(cero,"\t")
cerocinco <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution05MultilayerNetwork5112018.csv")
cerocinco <- strsplit(cerocinco,"\t")
uno <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution1MultilayerNetwork5112018.csv")
uno <- strsplit(uno,"\t")
unocinco <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution15MultilayerNetwork5112018.csv")
unocinco <- strsplit(unocinco,"\t")
dos <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution2MultilayerNetwork5112018.csv")
dos <- strsplit(dos,"\t")
doscinco <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution25MultilayerNetwork5112018.csv")
doscinco <- strsplit(doscinco,"\t")
tres <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution3MultilayerNetwork5112018.csv")
tres <- strsplit(tres,"\t")
trescinco <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution35MultilayerNetwork5112018.csv")
trescinco <- strsplit(trescinco,"\t")
cuatro <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution4MultilayerNetwork5112018.csv")
cuatro <- strsplit(cuatro,"\t")

In [5]:
lista_comunidades <- list(cero,cerocinco,uno,unocinco,dos,doscinco,tres,trescinco,cuatro)

In [6]:
lista_componentes <- list()
lista_sizes <- list()

##### ~16min run

In [7]:
Sys.time()

[1] "2023-03-20 11:15:18 CET"

In [8]:
for(k in 1:1000){
# Load and randomize severe diagnosis for each patient
patients2 <- read.table(file= "data/MolTi/Community_Analysis/originalSevereDiagnosis.csv",sep= ",",header= F)
patients2[,1] <- as.character(patients2[,1])
patients2[,2] <- as.character(patients2[,2])
patients2 <- cbind(patients2,0)
patients2[,3] <- sample(patients2[,2])

#Retrieve specific Compound Heterozygous Mutations for Severe & Non-severe groups
    
#Compound Heterozygous mutations
patients <- read.table(file= "data/InputGenes/compound_all_nov2018.csv")
patients[,1] <- as.character(patients[,1])
patients[,2] <- as.character(patients[,2])
patients <- split(patients[,2],patients[,1])
severepatients <- patients[which(patients2[,3]=="severe")] #Uncomment and comment the next one for randomizing
#severepatients <- patients[patients2[,1][which(patients2[,2]=="severe")]] ##Uncomment and comment the previous one for the original plot
notseverepatients <- patients[which(patients2[,3]=="notsevere")] #Uncomment and comment the next one for randomizing
#notseverepatients <- patients[patients2[,1][which(patients2[,2]=="notsevere")]] #Uncomment and comment the previous one for the original plot
severe<- unlist(severepatients)
names(severe) <- NULL
notsevere <- unlist(notseverepatients)
names(notsevere) <- NULL
severeonly <- setdiff(severe,notsevere)
notsevereonly <- setdiff(notsevere,severe)

severepatientsnames <- names(severepatients)
notseverepatientsnames <- names(notseverepatients)
    
# Retrieve specific Copy Number Variations for Severe & Non-severe groups
cnvdata <- read.csv("data/InputGenes/CNV_genes_for_iker.tsv",sep= "\t")
cnvdata[,"gene"] <- as.character(cnvdata[,"gene"])
genis <- c(cnvdata["gene"])$gene
genis <- mapIds(org.Hs.eg.db,keys = genis,column = "SYMBOL",keytype="ENSEMBL",multiVals = "first")
# Manual mapping for missing IDs
genis["ENSG00000005955"] <- "GGNBP2" 
genis["ENSG00000006114"] <- "SYNGR"
genis["ENSG00000108264"] <- "TADA2A"
genis["ENSG00000108270"] <- "AATF"
genis["ENSG00000108272"] <- "DHRS11"
genis["ENSG00000108278"] <- "ZNHI3"
genis["ENSG00000108753"] <- "HNF1B"
genis["ENSG00000129282"] <- "MRM1"
genis["ENSG00000132130"] <- "LHX1"
genis["ENSG00000141140"] <- "MYO19"
genis["ENSG00000141141"] <- "DDX52"
genis["ENSG00000161326"] <- "DUSP14"
genis["ENSG00000167230"] <- "C17orf78"
genis["ENSG00000174093"] <- "RP11-1407O15.2"
genis["ENSG00000184886"] <- "PIGW"
genis["ENSG00000197681"] <- "TBC1D3"
genis["ENSG00000203815"] <- "FAM231D"
genis["ENSG00000219492"] <- "RP11-1396O31.13"
genis["ENSG00000229924"] <- "FAM90A26"
genis["ENSG00000250913"] <- "USP17L23"
genis["ENSG00000268172"] <- "AL590452.1"
genis <- as.matrix(genis)
rownames(genis) <- NULL
genis <- genis[,1]
cnvdata <- cbind(cnvdata,genis)
severedata <- cnvdata[,c("gene","whole_gene",severepatientsnames,"genis")]
milddata <- cnvdata[,c("gene","whole_gene",notseverepatientsnames,"genis")]
    
severedata <- severedata[apply(X = severedata,MARGIN = 1,function(x) paste0(x[3:10],collapse='_')) != '2_2_2_2_2_2_2_2',]
milddata <- milddata[apply(X = milddata,MARGIN = 1,function(x) paste0(x[3:10],collapse='_')) != '2_2_2_2_2_2_2_2',]

rownames(severedata) <- NULL
rownames(milddata) <- NULL
severecnvs <- unique(as.character(severedata[,"genis"]))
mildcnvs <- unique(as.character(milddata[,"genis"]))
cnvsevereonly <- setdiff(severecnvs,mildcnvs)
cnvsnotsevereonly <- setdiff(mildcnvs,severecnvs)

# Load CMS Causal genes
cosa <- read.csv("data/InputGenes/cmsgenes.csv",header=FALSE) #http://www.musclegenetable.fr/4DACTION/Blob_groupe2
#cosa <- read.csv("data/InputGenes/genespaper.csv",header=FALSE) #Table PMID:30552423
cosa <- cosa[,1]
cosa <- as.character(cosa)
                           
# Join causal and mutated genes
# Setup depending on severity groups
if(phenotype == "severe"){
  algenes <- c(severeonly,cosa,cnvsevereonly) #Severe-Only Analysis 
}
if(phenotype == "notsevere"){
  algenes <- c(notsevereonly,cosa,cnvsnotsevereonly) #To get a Mild-Only  Analysis
}                        
# Solve some missing gene mappings
if("KIAA1919" %in% algenes){
  algenes <- replace(algenes,algenes== "KIAA1919","MFSD4B")
}
if("FAM188B" %in% algenes){
  algenes <- replace(algenes,algenes== "FAM188B","MINDY4")
}
if("C14orf159" %in% algenes){
  algenes <- replace(algenes,algenes== "C14orf159","DGLUCY")
}
if("C1orf168" %in% algenes){
  algenes <- replace(algenes,algenes== "C1orf168","FYB2")
}
if("IKBKAP" %in% algenes){
  algenes <- replace(algenes,algenes== "IKBKAP","ELP1")
}
if("SYNGR" %in% algenes){
  algenes <- replace(algenes,algenes== "SYNGR","SYNGR1")
}
if("ZNHI3" %in% algenes){
  algenes <- replace(algenes,algenes== "ZNHI3","ZNHIT3")
}
if("FAM231D" %in% algenes){
  algenes <- replace(algenes,algenes== "FAM231D","LINC01145")
}
if("SELO" %in% algenes){
  algenes <- replace(algenes,algenes== "SELO","SELENOO")
}
if("MFSD7" %in% algenes){
  algenes <- replace(algenes,algenes== "MFSD7","SLC49A3")
}

#Convert to entrez
algenes2 <- algenes
algenes <- mapIds(org.Hs.eg.db,keys = algenes,column = "ENTREZID",keytype="SYMBOL",multiVals = "first")
names(algenes) <- NULL                           

# Create a table with the community ID of each gene in 'genescausales' vector.
genescausales <- mapIds(org.Hs.eg.db,keys = algenes2,column = "ENTREZID",keytype="SYMBOL",multiVals = "first")
genescausales <- as.matrix(genescausales)
rownames(genescausales) <- NULL
genescausales <- genescausales[,1]
genescausales <- unlist(genescausales) #need this unlist as sometimes R does not create a c() vector with the previous line
disease <- genescausales
res <- matrix(nrow= length(disease),ncol= 9)
rownames(res) <- disease

#Remove genes not present on Multilayer network
curl <- lapply(as.list(genescausales),function(x) which(sapply(cuatro, function(y) x %in% y)))
res <- res[!unlist(lapply(curl,function(x) length(x)==0)),]

for(i in 1:9){
  current <- unlist(lapply(as.list(rownames(res)),function(x) which(sapply(lista_comunidades[[i]], function(y) x %in% y))))
  res[,i] <- current
} 
   
#  Generate shared community pertinance graph for randomly produced module                                                                          
melted_dist <- as.matrix(parallelDist(res,'hamming',threads = 7)*9)
diag(melted_dist) <- NA
melted_dist[lower.tri(melted_dist)] <- NA

rownames(melted_dist) <- mapIds(org.Hs.eg.db,keys = rownames(melted_dist),column = "SYMBOL",keytype="ENTREZID",multiVals = "first")
colnames(melted_dist) <- rownames(melted_dist)

melted_dist <- melt(melted_dist)
melted_dist <- melted_dist[melted_dist[,3]==0,] # Get pairs that are always together trough resolution
melted_dist <- na.omit(melted_dist)
#dim(melted_dist)                                                                         
g=simplify(graph_from_edgelist(as.matrix(melted_dist[,1:2]),directed = F))
componentes <- components(g)
lista_componentes[[k]] <- componentes$membership   
lista_sizes[[k]] <- componentes$csize
print(k)                                                                           
message(Sys.time())
}                                                                           

'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns


[1] 1


2023-03-20 11:15:20
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns


[1] 2


2023-03-20 11:15:21
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns


...

2023-03-20 11:39:55
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns


[1] 999


2023-03-20 11:39:56
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns


[1] 1000


2023-03-20 11:39:58


In [9]:
Sys.time()

[1] "2023-03-20 11:39:58 CET"

#### Check significance of finding Figure 4 modules upon severe / non-severe labelling shuffling

In [11]:
if(phenotype=='severe'){
    interesting_module <- c('PLEC','VCAN','TNXB','LOXL3','LAMB2','LRP4','HSPG2','LAMA5','COL13A1','TNC','USH2A','LAMA2','CHGB','AGRN','COL15A1')
} 
if(phenotype=='notsevere'){
    interesting_module <- c('LRP4','COL6A5','TLL2','MCAM','ADAMTS9','HMCN1','COL13A1','MSR1','LAMB2','LAMB4','LAMA5','ADAM28','PLEC','ITIH5','AGRN')
} 
maximum_per_iter <- c()
for(q in 1:length(lista_componentes)){
lista_cool <- split(names(lista_componentes[[q]]),lista_componentes[[q]])
num <- max(unlist(lapply(lista_cool,function(x) length(interesting_module[interesting_module %in% x]))))
maximum_per_iter <- c(maximum_per_iter,num)
}                

In [12]:
length(interesting_module) #15 genes (Severe)

[1] 15

In [13]:
table(maximum_per_iter)/1000

maximum_per_iter
    6     7     8     9    10    11    12    13    14 
0.048 0.084 0.200 0.206 0.200 0.159 0.073 0.022 0.008 

#### Finding a module with 13 or more of these genes is already significant. (p-value = 0.022) (Severe Module)

#### Finding a module with 14 or more of these genes is already significant. (p-value = 0.027) (Not-severe Module)